# Implementation of Decision Tree Algorithm from Scratch

In [1]:
import numpy as np
import pandas as pd
import time

In [2]:
class DecisionTree:
    def __init__(self):
        self.root = None
    # calculate the gini value for each group of every column/feature
    def gini_index(self, groups, classes):
        # count all the samples at split point
        n_instances = float(sum([len(group) for group in groups]))
        # sum weighted Gini index for each group
        gini = 0.0
        for group in groups:
            size = float(len(group))
            # avoid divide by zero
            if size==0:
                continue
            score = 0.0
            outcomes = [row[-1] for row in group]
            # score the group based on the score for each class
            for class_val in classes:
                p = outcomes.count(class_val) / size
                score += p*p
            # weight the group score by its relative size
            gini += (1.0 - score) * (size / n_instances)
        return gini

    # Split a dataset based on an attribute and an attribute value
    def test_split(self, index, value, dataset):
        left, right = list(), list()
        for row in dataset:
            if row[index] < value:
                left.append(row)
            else:
                right.append(row)
        return left, right

    # Select the best split point for a dataset
    def get_split(self, dataset):
        # get all the class values i.e the target values
        class_values = list(set(row[-1] for row in dataset))
        # declare best_index, best_value, best_score, best_group
        b_index, b_value, b_score, b_groups = float("inf"),float("inf"),float("inf"), None
        for index in range(len(dataset[0])-1):
            for row in dataset:
                groups = self.test_split(index, row[index], dataset)
                gini = self.gini_index(groups, class_values)
                if gini < b_score:
                    b_index, b_value, b_score, b_groups = index, row[index], gini, groups
        return {'index':b_index, 'value':b_value, 'groups':b_groups}
    
    # Create a terminal node value
    def to_terminal(self, group):
        outcomes = [row[-1] for row in group]
        return max(set(outcomes), key=outcomes.count)
    
    # Create child splits for a node or make terminal
    def split(self, node, min_size, max_depth, depth):
        left, right = node["groups"]
        # check for a no split
        if not left or not right:
            node["left"] = node["right"] = self.to_terminal(left+right)
            return
        # check for max depth
        if depth > max_depth:
            node["left"] = self.to_terminal(left)
            node["right"] = self.to_terminal(right)
            return
        # process left child
        if len(left) <= min_size:
            node["left"] = self.to_terminal(left)
        else:
            node["left"] = self.get_split(left)
            self.split(node["left"], min_size, max_depth, depth+1)
        # process right child
        if len(right) <= min_size:
            node["right"] = self.to_terminal(right)
        else:
            node["right"] = self.get_split(right)
            self.split(node["right"], min_size, max_depth, depth+1)
    # Build a decision tree
    def build_tree(self, train, max_depth, min_size):
    	self.root = self.get_split(train)
    	self.split(self.root, min_size, max_depth, 1)

    def predict(self, dataset):
        return [self.predict_(row, self.root) for row in dataset]
    
    def predict_(self, row, node):
        if row[node["index"]] < node["value"]:
            if isinstance(node["left"], dict):
                return self.predict_(row, node["left"])
            else:
                return node["left"]
        else:
            if isinstance(node["right"], dict):
                return self.predict_(row, node["right"])
            else:
                return node["right"]

    def accuracy_metric(self, dataset, pred_values):
        correct = sum([1 for ind, row in enumerate(dataset) if row[-1] == pred_values[ind]])
        return correct / float(len(dataset)) * 100.0

In [3]:
dataset = [[2.771244718,1.784783929,0],
	[1.728571309,1.169761413,0],
	[3.678319846,2.81281357,0],
	[3.961043357,2.61995032,0],
	[2.999208922,2.209014212,0],
	[7.497545867,3.162953546,1],
	[9.00220326,3.339047188,1],
	[7.444542326,0.476683375,1],
	[10.12493903,3.234550982,1],
	[6.642287351,3.319983761,1]]

max_depth = 2
min_size = 5
dt = DecisionTree()
dt.build_tree(dataset, max_depth, min_size)
pred_values = dt.predict(dataset)
accuracy = dt.accuracy_metric(dataset, pred_values)
print("Accuracy: {}".format(accuracy))

Accuracy: 100.0


Trying our class on a larger dataset

In [4]:
from csv import reader
# Load a CSV file
def load_csv(filename):
	file = open(filename, "rt")
	lines = reader(file)
	dataset = list(lines)
	return dataset

# Convert string column to float
def str_column_to_float(dataset, column):
	for row in dataset:
		row[column] = float(row[column].strip())

filename = r"C:\Users\Mohit\OneDrive\Desktop\ml\datasets\banknote+authentication\data_banknote_authentication.txt"
dataset = load_csv(filename)
# convert string attributes to integers
for i in range(len(dataset[0])):
	str_column_to_float(dataset, i)

max_depth = 5
min_size = 10
dt = DecisionTree()
dt.build_tree(dataset, max_depth, min_size)
pred_values = dt.predict(dataset)
accuracy = dt.accuracy_metric(dataset, pred_values)
print("Accuracy: {}".format(accuracy))

Accuracy: 99.56268221574344


## Working with the Titanic Dataset

In [5]:
# Load dataset
df = pd.read_csv(r"C:\Users\Mohit\OneDrive\Desktop\ml\datasets\Titanic-Dataset.csv")

# Select useful features
df = df[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Survived']]

# Convert categorical to numerical
df['Sex'] = df['Sex'].map({'male': 0, 'female': 1})

# Fill missing values (e.g., Age)
df["Age"] = df['Age'].fillna(df['Age'].median())

# Convert to list of lists format
dataset = df.values.tolist()

train_set = dataset[:600]
test_set = dataset[600:]

max_depth = 5
min_size = 10
dt = DecisionTree()
start = time.time()
dt.build_tree(train_set, max_depth, min_size)
pred_values = dt.predict(test_set)
accuracy = dt.accuracy_metric(test_set, pred_values)
print("Accuracy: {}".format(accuracy))
print("Time Taken: {}".format(time.time() - start))

Accuracy: 78.35051546391753
Time Taken: 0.8484780788421631


In [6]:
class DecisionTreeCF:
    '''
    Decision Tree with Categorical Features
    '''
    def __init__(self):
        self.root = None
    # calculate the gini value for each group of every column/feature
    def gini_index(self, groups, classes):
        # count all the samples at split point
        n_instances = float(sum([len(group) for group in groups]))
        # sum weighted Gini index for each group
        gini = 0.0
        for group in groups:
            size = float(len(group))
            # avoid divide by zero
            if size==0:
                continue
            score = 0.0
            outcomes = [row[-1] for row in group]
            # score the group based on the score for each class
            for class_val in classes:
                p = outcomes.count(class_val) / size
                score += p*p
            # weight the group score by its relative size
            gini += (1.0 - score) * (size / n_instances)
        return gini

    # Split a dataset based on an attribute and an attribute value
    def test_split(self, index, value, dataset):
        left, right = list(), list()
        for row in dataset:
            if row[index] != value:
                left.append(row)
            else:
                right.append(row)
        return left, right

    # Select the best split point for a dataset
    def get_split(self, dataset):
        # get all the class values i.e the target values
        class_values = list(set(row[-1] for row in dataset))
        # declare best_index, best_value, best_score, best_group
        b_index, b_value, b_score, b_groups = float("inf"),float("inf"),float("inf"), None
        for index in range(len(dataset[0])-1):
            # all unique values in the index/attribute
            unique_val = list(set(row[index] for row in dataset))
            for val in unique_val:
                groups = self.test_split(index, val, dataset)
                gini = self.gini_index(groups, class_values)
                if gini < b_score:
                    b_index, b_value, b_score, b_groups = index, val, gini, groups
        return {'index':b_index, 'value':b_value, 'groups':b_groups}
    
    # Create a terminal node value
    def to_terminal(self, group):
        outcomes = [row[-1] for row in group]
        return max(set(outcomes), key=outcomes.count)
    
    # Create child splits for a node or make terminal
    def split(self, node, min_size, max_depth, depth):
        left, right = node["groups"]
        # check for a no split
        if not left or not right:
            node["left"] = node["right"] = self.to_terminal(left+right)
            return
        # check for max depth
        if depth > max_depth:
            node["left"] = self.to_terminal(left)
            node["right"] = self.to_terminal(right)
            return
        # process left child
        if len(left) <= min_size:
            node["left"] = self.to_terminal(left)
        else:
            node["left"] = self.get_split(left)
            self.split(node["left"], min_size, max_depth, depth+1)
        # process right child
        if len(right) <= min_size:
            node["right"] = self.to_terminal(right)
        else:
            node["right"] = self.get_split(right)
            self.split(node["right"], min_size, max_depth, depth+1)
    # Build a decision tree
    def build_tree(self, train, max_depth, min_size):
    	self.root = self.get_split(train)
    	self.split(self.root, min_size, max_depth, 1)

    def predict(self, dataset):
        return [self.predict_(row, self.root) for row in dataset]
    
    def predict_(self, row, node):
        if row[node["index"]] != node["value"]:
            if isinstance(node["left"], dict):
                return self.predict_(row, node["left"])
            else:
                return node["left"]
        else:
            if isinstance(node["right"], dict):
                return self.predict_(row, node["right"])
            else:
                return node["right"]

    @staticmethod
    def accuracy_metric(dataset, pred_values):
        correct = sum([1 for ind, row in enumerate(dataset) if row[-1] == pred_values[ind]])
        return correct / float(len(dataset)) * 100.0

In [8]:
# Load dataset
df = pd.read_csv(r"C:\Users\Mohit\OneDrive\Desktop\ml\datasets\Titanic-Dataset.csv")

# Select useful features
df = df[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Survived']]

# Convert categorical to numerical
# df['Sex'] = df['Sex'].map({'male': 0, 'female': 1})

# Fill missing values (e.g., Age)
df["Age"] = df['Age'].fillna(df['Age'].median())

# Convert to list of lists format
dataset = df.values.tolist()

train_set = dataset[:600]
test_set = dataset[600:]

max_depth = 1
min_size = 10
start = time.time()
dt = DecisionTreeCF()
dt.build_tree(train_set, max_depth, min_size)
pred_values = dt.predict(test_set)
accuracy = dt.accuracy_metric(test_set, pred_values)
print("Accuracy: {}".format(accuracy))
print("Time Taken: {}".format(time.time() - start))

Accuracy: 77.66323024054984
Time Taken: 0.03952789306640625


## Using the sklearn model of Decision Tree

In [9]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

df['Sex'] = df['Sex'].map({'male': 0, 'female': 1})
X, y = df.iloc[:, :-1], df.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)

dtc = DecisionTreeClassifier()
dtc.fit(X_train, y_train)

DecisionTreeClassifier()

In [10]:
def accuracy_metric(y_true, y_pred):
    correct = sum([1 for ind, row in enumerate(y_true) if y_pred[ind] == y_true[ind]])
    return correct / float(len(y_pred)) * 100.0

In [11]:
y_pred = dtc.predict(X_test)
accuracy_metric(y_test.values, y_pred)

75.97765363128491